In [ ]:
!pip install pytelegrambotapi
!pip install keras

# подключаем модуль для Телеграма
import telebot
from telebot import types
from keras.models import load_model
from google.colab import drive
from keras.models import load_model
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
import numpy as np

In [ ]:
# Всё сохранение происходит на Гугл Драйв
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Указываем токен для доступа к боту
bot = telebot.TeleBot('6506142546:AAFC0GzG49H9gOZeUrzl13ZZQIYcrVJGPNs')

# Храним названия актуальных файлов
file_names = []

# Загружаем модель
model = load_model('my_model.h5')
model.compile()

In [ ]:
def determine_shape(file_name):
    img_width, img_height = 150, 150
    img = load_img(f'/content/bot_photo/{file_name}', target_size=(img_width, img_height))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    res = model.predict(images)[0]
    return res

In [ ]:
def get_message(prediction):
    return f"С вероятностями {prediction[0]*100}%, {prediction[1]*100}%, {prediction[2]*100}% на картинке изображены соответственно палки, кубы, шары"

In [ ]:
# Хэндлер текстовых сообщений
@bot.message_handler(content_types=['text'])
def get_text_messages(message):
  if message.text == "/start":
      # Обработчик нашего базового функционала
      bot.send_message(message.from_user.id,
      """Привет, я NanoPred. Сейчас я умею рассказывать о том, что вижу по твоим наночастицам.
      \n Обращаю твоё внимание: картинки должны иметь расширение 150x150
      \n Напиши /help, чтобы узнать больше и воспользоваться функционалом
      """)

  elif message.text == "/help":
      # Заводим первую кнопку для функции "картинка-тип"
      keyboard = types.InlineKeyboardMarkup()
      key_type_by_picture = types.InlineKeyboardButton(text='👀', callback_data='picture-type')
      keyboard.add(key_type_by_picture)

      bot.send_message(message.from_user.id, text=
      """
      Этот бот позволяет осуществлять определение типа структуры наночастиц по представленным фотографиям.
В будущем он научится предсказывать по входным параметрам синтеза фотографию синтезируемой наночастицы
Перед тем, как начать, не забудь загрузить фотографии. Затем нажми на 👀, и я скажу тебе, что вижу
      """, reply_markup=keyboard)

  else:
      bot.send_message(message.from_user.id, "Я не очень тебя понял. Напиши /help.")


# Сохранение картинок, которые приходят от пользователя
@bot.message_handler(content_types=['photo'])
def handle_docs_document(message):
    file_info = bot.get_file(message.photo[len(message.photo) - 1].file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    src = '/content/bot_photo/' + message.photo[1].file_id

    with open(src, 'wb') as new_file:
        new_file.write(downloaded_file)

    file_names.append(message.photo[1].file_id)
    bot.reply_to(message, "Фото добавлено")


# Обработчик нажатий на кнопки
@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    # Если нажали на кнопку с глазками "картинка -> тип", выводим предсказание типа
    if call.data == "picture-type":
        msg = "Если фотографии были добавлены до этого, я сделаю для них прогноз"
        bot.send_message(call.message.chat.id, msg)

        if not file_names:                                          # Нет файлов
            bot.send_message(call.message.chat.id, "Упс, нечего предсказывать, ты не отправил ни одной картинки. Попробуй ещё раз")
        else:                                                       # Есть файлы
            bot.send_message(call.message.chat.id, "Отлично. Сейчас сделаю")
            for i, picture in enumerate(file_names):
                # Здесь должно быть предсказание
                mes = get_message(determine_shape(picture))
                file_names.remove(picture)
                bot.send_message(call.message.chat.id, f"Картинка {i+1}: " + mes)
# Слушатель бота
bot.polling(none_stop=True, interval=0)


1/1 [==============================] - 0s 66ms/step
С вероятностями 0.0%, 0.0%, 100.0% на картинке изображены соответственно кубы, палки, шары
1/1 [==============================] - 0s 24ms/step
С вероятностями 0.0%, 0.0%, 100.0% на картинке изображены соответственно кубы, палки, шары
